In [12]:
import networkx as nx
import plotly.graph_objects as go
import os

def parse_dpd_file(file_path):
    nodes = {}
    edges = []

    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()
            if line.startswith("N:"):
                parts = line.split(maxsplit=3)
                node_id = parts[1]
                label = parts[2].strip('"')
                attributes = parts[3] if len(parts) > 3 else ""
                nodes[node_id] = {"label": label, "attributes": attributes}
            elif line.startswith("E:"):
                parts = line.split()
                source = parts[1]
                target = parts[2]
                edges.append((source, target))
    return nodes, edges

def plot_dependency_graph(file_path):
    nodes, edges = parse_dpd_file(file_path)
    G = nx.DiGraph()

    for node_id, data in nodes.items():
        G.add_node(node_id, **data)
    G.add_edges_from(edges)
    
    # Compute node positions
    pos = nx.spring_layout(G, seed=42, k=3 / (len(G.nodes) ** 0.5))
    
    # Prepare data for Plotly
    edge_x = []
    edge_y = []
    for edge in G.edges():
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])

    edge_trace = go.Scatter(
        x=edge_x,
        y=edge_y,
        line=dict(width=0.5, color="gray"),
        hoverinfo="none",
        mode="lines"
    )

    node_x = []
    node_y = []
    hover_text = []
    for node in G.nodes():
        x, y = pos[node]
        node_x.append(x)
        node_y.append(y)
        hover_text.append(
            f"ID: {node}<br>Label: {G.nodes[node]['label']}<br>Attributes: {G.nodes[node].get('attributes', 'None')}"
        )

    node_trace = go.Scatter(
        x=node_x,
        y=node_y,
        mode="markers",
        hoverinfo="text",
        text=hover_text,
        marker=dict(
            size=10,
            color="blue",
            line=dict(width=2)
        )
    )

    # Create the figure
    fig = go.Figure(data=[edge_trace, node_trace],
                    layout=go.Layout(
                        title="Dependency Graph",
                        titlefont_size=16,
                        showlegend=False,
                        hovermode="closest",
                        width=1600,
                        height=900, 
                        margin=dict(b=0, l=0, r=0, t=40),
                        xaxis=dict(showgrid=False, zeroline=False),
                        yaxis=dict(showgrid=False, zeroline=False),
                    ))
    
    fig.show()

# Replace with the path to your .dpd file
dpd_file_path = r"C:\Users\User\Documents\GitHub\autoformalization\src\temp\dgraph.dpd"
if os.path.exists(dpd_file_path):
    plot_dependency_graph(dpd_file_path)
else:
    print(f"Error: {dpd_file_path} not found.")
